In [1]:
import sys
sys.path.append("../")

In [2]:
import torch
import json
import yaml
import pickle
import argparse
# import nltk

# nltk.download('punkt')
from tqdm import tqdm
import os
import sys
import torch
import random
import logging
import numpy as np
from torch import nn
from torch.utils.data import DataLoader
from visdial.model import get_model
from visdial.data.dataset import VisDialDataset
from visdial.metrics import SparseGTMetrics, NDCG, scores_to_ranks
from visdial.utils.checkpointing import CheckpointManager, load_checkpoint_from_config
from visdial.utils import move_to_cuda
from options import get_training_config_and_args
from visdial.utils.model_utils import evaluate

# For batch processing 

## No finetuning

In [3]:
split = 'val'
decoder_type = 'disc'

couples = [('s13', '4'), 
           ('s15', '4'),
           ('s19', '4'),
           ('s21', '4'),
           ('s22', '4'),
           ('s23', '4'),
           ('s24', '4'),
           ('s25', '4'),
           ('s26', '4'),
          ]

for (name, ckpt) in couples:
    f1 = f"/home/quang/checkpoints/{name}/ranks/{split}/no_ft_ckpt_{ckpt}/opt_scores.json"
    f2 = f"/home/quang/checkpoints/{name}/ranks/{split}/no_ft_ckpt_{ckpt}/opt_scores.pkl"
    
    os.system(f"mv {f1} {f1.replace('opt_scores', 'disc')}")
    os.system(f"mv {f2} {f2.replace('opt_scores', 'disc')}")

# for (name, ckpt) in couples:
#     wpath = '/home/quang/checkpoints/{0}/checkpoint_{1}.pth'.format(name, ckpt)
#     cpath = '/home/quang/checkpoints/{0}/config.json'.format(name)
#     evaluate(cpath=cpath, wpath=wpath, split='val', device='cuda:1', ckpt_name=f"no_ft_ckpt_{ckpt}")

In [4]:
wpath = '/home/quang/checkpoints/abci/s41/checkpoint_13.pth'
cpath = '/home/quang/checkpoints/abci/s41/config.json'
evaluate(cpath=cpath, wpath=wpath, split='val', device='cuda:1', ckpt_name=f"no_ft_ckpt_13")

loading weights...
[val2018] Tokenizing questions...
[val2018] Tokenizing answers...
[val2018] Tokenizing captions...
genome_path None


100%|██████████| 2064/2064 [05:42<00:00,  5.88it/s]


Writing ranks to /home/quang/checkpoints/abci/s41/ranks/val/no_ft_ckpt_13/disc.json
r@1: 0.4893895387649536
r@5: 0.786482572555542
r@10: 0.8788275122642517
mean: 4.858866214752197
mrr: 0.6231963634490967
ndcg: 0.627185583114624


{'r@1': 0.4893895387649536,
 'r@5': 0.786482572555542,
 'r@10': 0.8788275122642517,
 'mean': 4.858866214752197,
 'mrr': 0.6231963634490967,
 'ndcg': 0.627185583114624}

## For test

In [3]:
split = 'test'
decoder_type = 'disc'

couples = [
           ('s13', '4'), 
           ('s15', '4'),
           ('s19', '4'),
           ('s21', '4'),
           ('s22', '4'),
           ('s23', '4'),
           ('s24', '4'),
           ('s25', '4'),
           ('s26', '4'),
           ('s13', '11'), 
           ('s15', '11'),
           ('s19', '11'),
           ('s21', '11'),
           ('s22', '11'),
           ('s23', '11'),
           ('s24', '11'),
           ('s25', '11'),
           ('s26', '11'),
          ]

for (name, ckpt) in couples:
    f1 = f"/home/quang/checkpoints/{name}/ranks/{split}/no_ft_ckpt_{ckpt}/opt_scores.json"
    f2 = f"/home/quang/checkpoints/{name}/ranks/{split}/no_ft_ckpt_{ckpt}/opt_scores.pkl"
    
    os.system(f"mv {f1} {f1.replace('opt_scores', 'disc')}")
    os.system(f"mv {f2} {f2.replace('opt_scores', 'disc')}")

# for (name, ckpt) in couples:
#     wpath = '/home/quang/checkpoints/{0}/checkpoint_{1}.pth'.format(name, ckpt)
#     cpath = '/home/quang/checkpoints/{0}/config.json'.format(name)
#     evaluate(cpath=cpath, wpath=wpath, split=split, device='cuda:1', ckpt_name=f"no_ft_ckpt_{ckpt}")

## With Finetuning

In [4]:
split = 'test'
decoder_type = 'disc'
ckpt = 0
wpath = '/home/quang/checkpoints/abci/n42/finetune/lr_1e-05/CosineLR/checkpoint_0.pth'
cpath = '/home/quang/checkpoints/abci/n42/config.json'
jpath = os.path.join(os.path.dirname(wpath), 'ranks', split, f'no_ft_ckpt_{ckpt}', f'{decoder_type}.json')
ppath = os.path.join(os.path.dirname(wpath), 'ranks', split, f'no_ft_ckpt_{ckpt}', f'{decoder_type}.pkl')
evaluate(cpath, wpath, split=split, device='cuda:0', ckpt_name='ft_ckpt_0')

loading weights...
[test2018] Tokenizing questions...
[test2018] Tokenizing answers...
[test2018] Tokenizing captions...
genome_path None


100%|██████████| 8000/8000 [15:25<00:00,  8.55it/s]


Writing ranks to /home/quang/checkpoints/abci/n42/finetune/lr_1e-05/CosineLR/ranks/test/ft_ckpt_0/disc.json


{}

In [ ]:
split = 'test'
decoder_type = 'disc'

couples = [('s19', 11), ('s21', 11), ('s22', 10), ('s23', 10), ('s24', 11), ('s26', 11)]

for (name, ckpt) in couples:
    wpath = '/home/quang/checkpoints/{0}/checkpoint_{1}.pth'.format(name, ckpt)
    cpath = '/home/quang/checkpoints/{0}/train_simple_12epochs_{0}.yml'.format(name)
    jpath = os.path.join(os.path.dirname(wpath), 'ranks', split, f'no_ft_ckpt_{ckpt}', f'{decoder_type}.json')
    ppath = os.path.join(os.path.dirname(wpath), 'ranks', split, f'no_ft_ckpt_{ckpt}', f'{decoder_type}.pkl')
    evaluate(cpath, wpath, jpath, ppath, split='val', device='cuda:0')

In [33]:
for ckpt in [1, 4, 5, 6]:
    decoder_type = 'disc'
    split = 'test'

    weight_paths = [
        "/media/local_workspace/quang/checkpoints/visdial/CVPR/train_simple/lr001/12_epochs/s07_simple_branch/checkpoint_11.pth",
        "/media/local_workspace/quang/checkpoints/visdial/CVPR/train_simple/lr001/12_epochs/s08_simple_branch/checkpoint_11.pth",
        "/media/local_workspace/quang/checkpoints/visdial/CVPR/train_simple/lr001/12_epochs/s09_simple_branch/checkpoint_last.pth",
        "/media/local_workspace/quang/checkpoints/visdial/CVPR/train_simple/lr001/12_epochs/s10_simple_branch/checkpoint_11.pth",
        "/media/local_workspace/quang/checkpoints/visdial/CVPR/train_simple/lr001/12_epochs/s11_simple_branch/checkpoint_11.pth"
    ]

    config_paths = [
        "configs/train_simple_s07_simple_branch.yml",
        "configs/train_simple_s08_simple_branch.yml",
        "configs/train_simple_s09_simple_branch.yml",
        "configs/train_simple_s10_simple_branch.yml",
        "configs/train_simple_s11_simple_branch.yml"    
    ]
    json_paths = [os.path.join(os.path.dirname(path), 'ranks', split, f'ft_ckpt_{ckpt}', f'{decoder_type}.json') for path in weight_paths]
    pick_paths = [os.path.join(os.path.dirname(path), 'ranks', split, f'ft_ckpt_{ckpt}', f'{decoder_type}.pkl') for path in weight_paths]


    for cpath, wpath, jpath, ppath in zip(config_paths, weight_paths, json_paths, pick_paths):
        evaluate(cpath, wpath, jpath, ppath, split=split, device="cuda")

loading weights...
[test2018] Tokenizing questions...
[test2018] Tokenizing answers...
[test2018] Tokenizing captions...


100%|██████████| 8000/8000 [10:55<00:00, 12.21it/s]


Writing ranks to /media/local_workspace/quang/checkpoints/visdial/CVPR/train_simple/lr001/12_epochs/s07_simple_branch/ranks/test/ft_ckpt_1/disc.json
loading weights...
[test2018] Tokenizing questions...
[test2018] Tokenizing answers...
[test2018] Tokenizing captions...


100%|██████████| 8000/8000 [10:59<00:00,  9.72it/s]


Writing ranks to /media/local_workspace/quang/checkpoints/visdial/CVPR/train_simple/lr001/12_epochs/s08_simple_branch/ranks/test/ft_ckpt_1/disc.json
loading weights...
[test2018] Tokenizing questions...
[test2018] Tokenizing answers...
[test2018] Tokenizing captions...


...

Writing ranks to /media/local_workspace/quang/checkpoints/visdial/CVPR/train_simple/lr001/12_epochs/s09_simple_branch/ranks/test/ft_ckpt_6/disc.json
loading weights...
[test2018] Tokenizing questions...
[test2018] Tokenizing answers...
[test2018] Tokenizing captions...


100%|██████████| 8000/8000 [11:36<00:00, 11.48it/s]


Writing ranks to /media/local_workspace/quang/checkpoints/visdial/CVPR/train_simple/lr001/12_epochs/s10_simple_branch/ranks/test/ft_ckpt_6/disc.json
loading weights...
[test2018] Tokenizing questions...
[test2018] Tokenizing answers...
[test2018] Tokenizing captions...


100%|██████████| 8000/8000 [11:38<00:00, 12.84it/s]


Writing ranks to /media/local_workspace/quang/checkpoints/visdial/CVPR/train_simple/lr001/12_epochs/s11_simple_branch/ranks/test/ft_ckpt_6/disc.json


## For Finetuning  

In [9]:
for ckpt in [1, 4, 5, 6]:
    decoder_type = 'disc'
    split = 'test'

    weight_paths = [
        "/media/local_workspace/quang/checkpoints/visdial/CVPR/train_simple/lr001/12_epochs/s06_simple_branch/finetune/lr_5e-05/CosineLR/checkpoint_{}.pth",
        "/media/local_workspace/quang/checkpoints/visdial/CVPR/train_simple/lr001/12_epochs/s07_simple_branch/finetune/lr_5e-05/CosineLR/checkpoint_{}.pth",
        "/media/local_workspace/quang/checkpoints/visdial/CVPR/train_simple/lr001/12_epochs/s08_simple_branch/finetune/lr_5e-05/CosineLR/checkpoint_{}.pth",
        "/media/local_workspace/quang/checkpoints/visdial/CVPR/train_simple/lr001/12_epochs/s09_simple_branch/finetune/lr_5e-05/CosineLR/checkpoint_{}.pth",
        "/media/local_workspace/quang/checkpoints/visdial/CVPR/train_simple/lr001/12_epochs/s10_simple_branch/finetune/lr_5e-05/CosineLR/checkpoint_{}.pth",
        "/media/local_workspace/quang/checkpoints/visdial/CVPR/train_simple/lr001/12_epochs/s11_simple_branch/finetune/lr_5e-05/CosineLR/checkpoint_{}.pth"
    ]

    config_paths = [
        "configs/train_simple_s06_simple_branch.yml",
        "configs/train_simple_s07_simple_branch.yml",
        "configs/train_simple_s08_simple_branch.yml",
        "configs/train_simple_s09_simple_branch.yml",
        "configs/train_simple_s10_simple_branch.yml",
        "configs/train_simple_s11_simple_branch.yml"    
    ]
    weight_paths = [p.format(str(ckpt)) for p in weight_paths]
    json_paths = [os.path.join(os.path.dirname(path), 'ranks', split, f'ft_ckpt_{ckpt}', f'{decoder_type}.json') for path in weight_paths]
    pick_paths = [os.path.join(os.path.dirname(path), 'ranks', split, f'ft_ckpt_{ckpt}', f'{decoder_type}.pkl') for path in weight_paths]


    for cpath, wpath, jpath, ppath in zip(config_paths, weight_paths, json_paths, pick_paths):
        evaluate(cpath, wpath, jpath, ppath, split=split, device="cuda")
        


loading weights...
[test2018] Tokenizing questions...
[test2018] Tokenizing answers...
[test2018] Tokenizing captions...



100%|██████████| 8000/8000 [03:07<00:00, 42.64it/s]

Writing ranks to /media/local_workspace/quang/checkpoints/visdial/CVPR/train_simple/lr001/12_epochs/s06_simple_branch/finetune/lr_5e-05/CosineLR/ranks/test/ckpt_1/disc.json
loading weights...
[test2018] Tokenizing questions...
[test2018] Tokenizing answers...
[test2018] Tokenizing captions...



100%|██████████| 8000/8000 [03:13<00:00, 41.37it/s]

Writing ranks to /media/local_workspace/quang/checkpoints/visdial/CVPR/train_simple/lr001/12_epochs/s07_simple_branch/finetune/lr_5e-05/CosineLR/ranks/test/ckpt_1/disc.json
loading weights...
[test2018] Tokenizing questions...
[test2018] Tokenizing answers...
[test2018] Tokenizing captions...


...

Writing ranks to /media/local_workspace/quang/checkpoints/visdial/CVPR/train_simple/lr001/12_epochs/s09_simple_branch/finetune/lr_5e-05/CosineLR/ranks/test/ckpt_6/disc.json
loading weights...
[test2018] Tokenizing questions...
[test2018] Tokenizing answers...
[test2018] Tokenizing captions...



100%|██████████| 8000/8000 [03:10<00:00, 41.89it/s]

Writing ranks to /media/local_workspace/quang/checkpoints/visdial/CVPR/train_simple/lr001/12_epochs/s10_simple_branch/finetune/lr_5e-05/CosineLR/ranks/test/ckpt_6/disc.json
loading weights...
[test2018] Tokenizing questions...
[test2018] Tokenizing answers...
[test2018] Tokenizing captions...



100%|██████████| 8000/8000 [03:11<00:00, 41.76it/s]

Writing ranks to /media/local_workspace/quang/checkpoints/visdial/CVPR/train_simple/lr001/12_epochs/s11_simple_branch/finetune/lr_5e-05/CosineLR/ranks/test/ckpt_6/disc.json
